Part #1 - Some Global Functions

Function A - Importing Market Data

In [1]:
# n = Name of the excel file
# s = Sheet Name within the excel file
# r = Range of data which needs to be imported
# d = Data object that needs to be sent while exporting back

def import_excel(n,s,r):
    import xlwings as xw
    wb = xw.Book(n)
    sheet = wb.sheets[s]
    data = sheet.range(r).value
    return data 

def export_excel(n,s,r,d):
    import xlwings as xw
    wb = xw.Book(n)
    sheet = wb.sheets[s]
    sheet.range(r).value = d

Function B - MFBD Holiday Calendar

In [2]:
#dt = Date on which operation is to be performed
#cal1 = Country of Calendar 1, For USA pass argument US
#cal2 = Country of Calendar 2, For India pass argument IN .. This is an optional argument
# At present this function suports a max of 2 calendars only for MFBD

def mfbd(dt,cal1, cal2="NIL"):
    from datetime import date
    from datetime import timedelta
    import holidays
    direction = 1
    calendar1 = holidays.country_holidays(cal1)
    if cal2 == "NIL":
        while dt.weekday() > 4 or dt in calendar1:
            if dt.month == (dt+timedelta(days=1)).month - 1:
                direction = -1
            dt = dt + timedelta(days=1)*direction 
    else:
        calendar2 = holidays.country_holidays(cal2)
        while dt.weekday() > 4 or dt in calendar1 or dt in calendar2:
            if dt.month == (dt+timedelta(days=1)).month - 1:
                direction = -1
            dt = dt + timedelta(days=1)*direction 
    return dt

Function C - Curve Bootstrapping Function

Gives you a set of Discount Factors and Zero Rates for a given IRS curve (USD in this case)

In [169]:
def bootstrapir(data,i=-1,p=1):
    import pandas as pd
    from datetime import date,timedelta
    from dateutil.relativedelta import relativedelta
    df = pd.DataFrame(data)
    df.columns=['Tenor','Unit','Rate']
    #Set Pricing Date
    PDate = date(2025,5,19)  #You can set it to desired pricing date
    SDate = mfbd(PDate + timedelta(days=2),'US') #Calculating the Spot Start Date
    
    i = int(i)
    if i>-1:
        df.iat[i,2]=df.iat[i,2]+0.0001*p
    # Compute the Maturity Dates and Payment Dates for each tenor (supports weeks and months only as of now)
    # Assuming 2 Days Payment Delay for Standard SOFR Swaps as per current market convention
    for index,row in df.iterrows():
        if df.at[index,'Unit']== "w":
            df.at[index,'MDate']=mfbd(SDate+timedelta(weeks=df.at[index,'Tenor']),'US')
        else:
            df.at[index,'MDate']=mfbd(SDate+relativedelta(months=df.at[index,'Tenor']),'US')
        df.at[index,'PDate']=mfbd(df.at[index,'MDate']+timedelta(days=2),'US')
    df['DC']=(df['PDate']-SDate)/timedelta(days=1)/360
    df['Days']=(df['PDate']-SDate)/timedelta(days=1)
    df['DF']=1/(1+df['Rate']*df['DC'])
   
    return df

Function D - Compare DFs against an alternate benchmark like Bloomberg

![title][def]

[def]: SOFRBBG.png

In [4]:
def compare(df):
    import pandas as pd
    bbgdata = import_excel('Market Data.xlsx','USDIRSShort','h3:h17')
    bbg = pd.DataFrame(bbgdata)
    bbg.columns=['BBGDF']
    comp = df[['Tenor','Unit','PDate','DF']]
    comp['BBGDF'] = bbg['BBGDF'] #Need to find a better method to append columns as this is giving a warning 
    comp['DF Gap(bps)']=(comp['BBGDF']-comp['DF'])*10000/comp['BBGDF']
    print(comp)

In [5]:
data = import_excel('Market Data.xlsx','USDIRSShort','A3:C17')
df=bootstrapir(data)
compare(df)

    Tenor Unit       PDate        DF     BBGDF  DF Gap(bps)
0     1.0    w  2025-05-30  0.998924  0.998924    -0.000714
1     2.0    w  2025-06-06  0.998086  0.998086     0.000717
2     3.0    w  2025-06-13  0.997249  0.997249     0.000693
3     1.0    m  2025-06-25  0.995815  0.995815     0.003759
4     2.0    m  2025-07-23  0.992482  0.992482    -0.002813
5     3.0    m  2025-08-25  0.988606  0.988840     2.362592
6     4.0    m  2025-09-24  0.985123  0.985120    -0.026390
7     5.0    m  2025-10-23  0.981846  0.981841    -0.049781
8     6.0    m  2025-11-24  0.978317  0.978423     1.084186
9     7.0    m  2025-12-24  0.975103  0.975094    -0.092741
10    8.0    m  2026-01-23  0.971971  0.971959    -0.122421
11    9.0    m  2026-02-25  0.968613  0.968599    -0.147335
12   10.0    m  2026-03-25  0.965816  0.965799    -0.171059
13   11.0    m  2026-04-23  0.962992  0.962973    -0.198750
14   12.0    m  2026-05-26  0.959802  0.960094     3.043136


/var/folders/5g/4br4skc15dlb750j33xn06rh0000gn/T/ipykernel_768/2177254367.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp['BBGDF'] = bbg['BBGDF'] #Need to find a better method to append columns as this is giving a warning
/var/folders/5g/4br4skc15dlb750j33xn06rh0000gn/T/ipykernel_768/2177254367.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp['DF Gap(bps)']=(comp['BBGDF']-comp['DF'])*10000/comp['BBGDF']


We see that the Discount Factors Generated are very close except for the 3 months and 12 months point. 

Function E - FX Curve Bootstrap Function 

This takes as an input short dated fx spot + swap points (You can do it using outrights as well but either approach works)

Please note that INR Onshore short term fx swap curve trades on the month end instead of rollings which is the general convention globally and hence I have chosen the most liquid points for building the curves. 

Using the rolling points may be more convenient but you do end up losing a bit of precision which is not desirable (However, the NDF points are more liquid on rolling basis in line with international convention)

In [170]:
def bootstrapfx(data,usdf,i=-1,p=1):
    import pandas as pd
    import numpy as np
    from datetime import date,timedelta, datetime, time
    df=pd.DataFrame(data)
    df.columns=['Date','Points']
    spotrate=85.50 #I have taken the liberty of putting the spot rate here in the code directly but you can import it from excel / any other source
    df['OR']=spotrate + df['Points']/100
    #Set Pricing Date
    PDate = date(2025,5,19)  #You can set it to desired pricing date
    SDate = mfbd(PDate + timedelta(days=2),'US','IN') #Calculating the Spot Start Date
    #Add 1 as Discount Factor for Spot Date in the Discount Factors Data Frame for US Rates 
    usdf.loc[-1] = [0.0,"w",0.0,SDate,SDate,0,0,1.00000]
    usdf.index = usdf.index+1
    usdf = usdf.sort_index()
    #Copy equivalent USD DFs into the FX based DF table
    #This will require us to interpolate and we are creating a float column as we cant interpolate on dates directly
    MyTime = time(0,0,0)
    df['Days']=(df['Date']-datetime.combine(SDate,MyTime))/timedelta(days=1)
    usdf['PDate'] = usdf['PDate'].astype("datetime64[ns]")
    usdf['Days']=(usdf['PDate']-datetime.combine(SDate,MyTime))/timedelta(days=1)
    df['USDF']=np.interp(df['Days'],usdf['Days'],usdf['DF'])
    df['INRDF']=df['USDF']*spotrate/df['OR']
    df['INRYld']=(1/df['INRDF']-1)*365/df['Days']
    #This step overwrites the DFs and the Outrights with perturbed values if any
    i = int(i)
    if i>-1:
        df.iat[i,6]=df.iat[i,6]+0.0001*p
    df['INRDF']=1/(1+df['INRYld']*df['Days']/365)
    df['OR']=spotrate *df['USDF']/df['INRDF']
    return df



Please note that the forwards data is upto 373 days and the USD IRS data is for a slightly shorter tenor creating a potential issue if you try to price trades accurately between those 2 days (3 days in this example between day 370 and 373. I have just ignored this for the moment)

Function F - This will price a portfolio of short dated fx outright forwards and compute their NPV

In [184]:
#Variable i controls the US Curve points perturb
#Variable j controls the INR FX Curve perturb
#gd is the variable which controls the generate dependent curve behaviour

def priceFX(i=-1,j=-1,p=1,gd=1):
    import pandas as pd
    import numpy as np
    from datetime import date,timedelta, datetime, time
    spotrate=85.50
    #Generate US Curve
    data = import_excel('Market Data.xlsx','USDIRSShort','A3:C17')
    usdf=bootstrapir(data,i,p)
    usdforig=bootstrapir(data)
    #Generate INR FX Curve
    data = import_excel('Market Data.xlsx','INRFX','A3:B15')
    if gd==1:
        inrdf=bootstrapfx(data,usdf,j,p)
    else:
        inrdf=bootstrapfx(data,usdforig,j,p)

    data = import_excel('TradePrice.xlsx','ShortINRFX','B3:D4') #Import market data
    fxport = pd.DataFrame(data)
    fxport.columns=['Notional','Date','Rate']
    usdf=usdf.sort_index()
    PDate = date(2025,5,19)  #You can set it to desired pricing date
    SDate = mfbd(PDate + timedelta(days=2),'US','IN') #Calculating the Spot Start Date
    MyTime = time(0,0,0)
    fxport['Days']=(fxport['Date']-datetime.combine(SDate,MyTime))/timedelta(days=1)
    fxport['MktRate']=np.interp(fxport['Days'],inrdf['Days'],inrdf['OR'])
    fxport['INRDF']=np.interp(fxport['Days'],inrdf['Days'],inrdf['INRDF'])
    fxport['USDDF']=np.interp(fxport['Days'],usdf['Days'],usdf['DF'])
    fxport['INRNPV']=fxport['Notional']*(fxport['MktRate']-fxport['Rate'])*fxport['INRDF']
    fxport['INRLegPV']=fxport['Notional']*fxport['Rate']*-1*fxport['INRDF']
    fxport['USDLegPV']=fxport['Notional']*fxport['USDDF']
    #print(usdf)
    #print(inrdf)
    #print(fxport)
    #print(fxport['INRLegPV'].sum())
    #return fxport['INRNPV'].sum()
    return fxport['INRLegPV'].sum()+fxport['USDLegPV'].sum()*spotrate
    

In [185]:
NPV = priceFX()
print("Current Portfolio NPV : ",round(NPV,2))

Current Portfolio NPV :  4162199.1


Now lets price and generate currency wise sensitivity for this portfolio of trades

Approach #1 - Without generate dependents - Most traders in India tend to use this. There is no right or wrong answer between this approach and with generate dependents. A more detailed explanation is out of scope as only someone who has truly run long and short fx books would understand this. I will try and see if I can elaborate further later. Reach out to me directly on pushkargondane@gmail.com if you would like to chat about it. 

In a nutshell the difference related to if a curve is regenerated if its base or underlying curve is perturbed (In this case INR curve is derived out of the USD Curve and everytime we bump the USD Curve, the INR curve is not getting bumped subsequently aka not getting regenerated => No generation of  dependent curves)

In [192]:

NPV = priceFX()
print("Current Portfolio NPV : ",round(NPV,2))

#Lets start with USD Dv01 buckets first
Dv01 = 0
for i in range(15):
    NewPV = priceFX(i,-1,1,0)
    print("USD Bucket ",i+1 ,":" , round(NewPV - NPV,2))

    Dv01 = Dv01 + NewPV - NPV
print ("Total USD Dv01 : ", round(Dv01,2))

#Lets now move to INR Dv01 buckets
Dv01 = 0
for i in range(13):
    NewPV = priceFX(-1,i)
    print("INR Bucket ",i+1 ,":" , round(NewPV - NPV,2))

    Dv01 = Dv01 + NewPV - NPV
print ("Total INR Dv01 : ", round(Dv01,2))

Current Portfolio NPV :  45921068.1
USD Bucket  1 : 0.0
USD Bucket  2 : 0.0
USD Bucket  3 : 0.0
USD Bucket  4 : 0.0
USD Bucket  5 : 0.0
USD Bucket  6 : 0.0
USD Bucket  7 : 0.0
USD Bucket  8 : 0.0
USD Bucket  9 : 0.0
USD Bucket  10 : 0.0
USD Bucket  11 : -41982.1
USD Bucket  12 : -15123.97
USD Bucket  13 : 0.0
USD Bucket  14 : -58473.52
USD Bucket  15 : -17169.98
Total USD Dv01 :  -132749.57
INR Bucket  1 : 0.0
INR Bucket  2 : 0.0
INR Bucket  3 : 0.0
INR Bucket  4 : 0.0
INR Bucket  5 : 0.0
INR Bucket  6 : 0.0
INR Bucket  7 : 0.0
INR Bucket  8 : 0.0
INR Bucket  9 : 52055.17
INR Bucket  10 : 2121.35
INR Bucket  11 : 0.0
INR Bucket  12 : 70960.44
INR Bucket  13 : 0.0
Total INR Dv01 :  125136.96


Approach #2 - This is the proper generate dependents algorithm wherein the USD Curve is generated and any bumps in the USD Curve create a bump in the curves dependent on them aka INR curve in this case. Note that the USD Sensitivity is not completely 0 across the board in this case. 

In [193]:
NPV = priceFX()
print("Current Portfolio NPV : ",round(NPV,2))

#Lets start with USD Dv01 first
Dv01 = 0
for i in range(15):
    NewPV = priceFX(i)
    print("USD Bucket ",i+1 ,":" , round(NewPV - NPV,2))

    Dv01 = Dv01 + NewPV - NPV
print ("Total USD Dv01 : ", round(Dv01,2))

#Lets start with USD Dv01 first
Dv01 = 0
for i in range(13):
    NewPV = priceFX(-1,i)
    print("INR Bucket ",i+1 ,":" , round(NewPV - NPV,2))

    Dv01 = Dv01 + NewPV - NPV
print ("Total INR Dv01 : ", round(Dv01,2))

Current Portfolio NPV :  45921068.1
USD Bucket  1 : 0.0
USD Bucket  2 : 0.0
USD Bucket  3 : 0.0
USD Bucket  4 : 0.0
USD Bucket  5 : 0.0
USD Bucket  6 : 0.0
USD Bucket  7 : 0.0
USD Bucket  8 : 0.0
USD Bucket  9 : 0.0
USD Bucket  10 : 0.0
USD Bucket  11 : -933.0
USD Bucket  12 : -668.21
USD Bucket  13 : 169.44
USD Bucket  14 : -1784.53
USD Bucket  15 : -524.0
Total USD Dv01 :  -3740.3
INR Bucket  1 : 0.0
INR Bucket  2 : 0.0
INR Bucket  3 : 0.0
INR Bucket  4 : 0.0
INR Bucket  5 : 0.0
INR Bucket  6 : 0.0
INR Bucket  7 : 0.0
INR Bucket  8 : 0.0
INR Bucket  9 : 52055.17
INR Bucket  10 : 2121.35
INR Bucket  11 : 0.0
INR Bucket  12 : 70960.44
INR Bucket  13 : 0.0
Total INR Dv01 :  125136.96
